In [1]:
from ROOT import TMVA, TFile, TTree, TCut
from subprocess import call
from os.path import isfile
from array import array
import torch
from torch import nn
import numpy as np
import ROOT

Welcome to JupyROOT 6.28/00


In [ ]:
TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize()
 
output = TFile.Open('TMVAtestvbf.root', 'RECREATE')


factory = TMVA.Factory('TMVAClassification', output,
                       '!V:ROC:!Silent:Color:DrawProgressBar:Transformations=D,G:AnalysisType=Classification')

In [ ]:
sigfile=TFile.Open('signal_tree.root')
backfile=TFile.Open('background_tree.root')
sigtrain = sigfile.Get('EventTree')
backtrain = backfile.Get('EventTree')
dataloader = TMVA.DataLoader('dataset') #load dataset, name it dataset
dataloader.AddVariable('pt')
dataloader.AddVariable('eta')
dataloader.AddVariable('time30') #change this line based on desired time smearing
dataloader.AddSignalTree(sigtrain, 1.0)  #adds data from signal tree to dataset
dataloader.AddBackgroundTree(backtrain, 1.0) #adds data from background tree to dataset
dataloader.PrepareTrainingAndTestTree(TCut(''),
                                      'nTrain_Signal=464627:nTrain_Background=438219:SplitMode=Random:NormMode=NumEvents:!V') 


In [ ]:
model = nn.Sequential()  #sequential is used when creating models so that the output of the first module is passed to the next and so on
model.add_module('linear_1', nn.Linear(in_features=3, out_features=64))  #applies a linear transformation to incoming data
model.add_module('relu', nn.ReLU()) #activation function, will output input if it is positive or output 0 otherwise, this applies the function to each element
model.add_module('linear_2', nn.Linear(in_features=64, out_features=2)) 
model.add_module('softmax', nn.Softmax(dim=1)) #rescales input so that output elements are in range [0,1] and sum to 1

In [ ]:
# Construct loss function and Optimizer.
loss = torch.nn.MSELoss() #measures mean squared error between each element in input x and target y
optimizer = torch.optim.SGD #implements stochastic gradient descent

In [ ]:
# Define train function
def train(model, train_loader, val_loader, num_epochs, batch_size, optimizer, criterion, save_best, scheduler):
    trainer = optimizer(model.parameters(), lr=0.01) 
    schedule, schedulerSteps = scheduler #a schedule makes the learning rate adaptive to the gradient descent optimization
    best_val = None
 
    for epoch in range(num_epochs): #data is fed in bunches, each epoch contains many bunches 
        # Training Loop
        # Set to train mode
        model.train() 
        running_train_loss = 0.0
        running_val_loss = 0.0
        for i, (X, y) in enumerate(train_loader):  
            trainer.zero_grad() #zero the gradients for each batch 
            output = model(X) 
            train_loss = criterion(output, y) #computes the loss, criterion is the specific loss we want to minimize 
            train_loss.backward() #calculate the gradients
            trainer.step() #adjust the learning weights
 
            # print train statistics
            running_train_loss += train_loss.item()
            if i % 32 == 31:    # print every 32 mini-batches
                print("[{}, {}] train loss: {:.3f}".format(epoch+1, i+1, running_train_loss / 32))
                running_train_loss = 0.0
 
        if schedule:
            schedule(optimizer, epoch, schedulerSteps) #updates learning rate in each epoch
 
        # Validation Loop
        # Set to eval mode
        model.eval() 
        with torch.no_grad():
            for i, (X, y) in enumerate(val_loader): #our validation dataset which is separate from the training set
                output = model(X) #pass data through model
                val_loss = criterion(output, y) #find the loss
                running_val_loss += val_loss.item() #calculate loss
 
            curr_val = running_val_loss / len(val_loader) 
            if save_best:
               if best_val==None:
                   best_val = curr_val  #track best performance
               best_val = save_best(model, curr_val, best_val)
 
            # print val statistics per epoch
            print("[{}] val loss: {:.3f}".format(epoch+1, curr_val))
            running_val_loss = 0.0
 
    print("Finished Training on {} Epochs!".format(epoch+1))
 
    return model

In [ ]:
# Define predict function
def predict(model, test_X, batch_size=32):
    # Set to eval mode
    model.eval() 
 
    test_dataset = torch.utils.data.TensorDataset(torch.Tensor(test_X)) 
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  #load testing data
 
    predictions = []
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            X = data[0] 
            outputs = model(X)
            predictions.append(outputs)
        preds = torch.cat(predictions) #concatenates the sequence of prediction tensors
 
    return preds.numpy()

In [ ]:
load_model_custom_objects = {"optimizer": optimizer, "criterion": loss, "train_func": train, "predict_func": predict} #creates a dictionary for us to specify our parameters within the TMVA interface 
 
 
# Store model to file
# Convert the model to torchscript before saving
m = torch.jit.script(model)
torch.jit.save(m, "modelClassification.pt")
print(m)
 

In [ ]:
#Book the TMVA method we want
factory.BookMethod(dataloader, TMVA.Types.kPyTorch, 'Train and test ttbar',
                   'H:!V:VarTransform=D,G:FilenameModel=modelClassification.pt:FilenameTrainedModel=trainedModelClassification.pt:NumEpochs=20:BatchSize=32')

In [ ]:
# Run training, test and evaluation
factory.TrainAllMethods()
factory.TestAllMethods()
factory.EvaluateAllMethods()

In [ ]:
roc = factory.GetROCCurve(dataloader, 'Train and test ttbar') 
canvas = ROOT.TCanvas("Canvas","a first way to plot a variable",800,600)
canvas.SetGrid()
roc.Draw()